# Bug Helper
> A friendly bug interceptor

In [ ]:
# default_exp bug

In [5]:
# export
from IPython.core.ultratb import AutoFormattedTB
from traceback import format_exc
from datetime import datetime
from forgebox.html import list_group, list_group_kv, HTML
import html
import json
import base64
from jinja2 import Template
from unpackai.utils import STATIC

In [18]:
# export
itb = AutoFormattedTB(mode = 'Plain', tb_offset = 1)

def render_download_button(
    bytes_data:bytes,
    filename: str,
    description: str="Download",
    color:str = "default"):

    """
    Loads data from buffer into base64 payload
        embedded into a HTML button.
    Recommended for small files only.

    bytes_data: open file object ready for reading.
        A file like object with a read method.
    filename:    str
        The name when it is downloaded.
    description: str
        The text that goes into the button.

    """
    payload = base64.b64encode(bytes_data).decode()
    
    with open(STATIC/"html"/"download_button.html","r") as f:
        temp = Template(f.read())
        
    download_button = temp.render(
        filename=filename,
        payload=payload,
        color=color,
        description=description)
    return download_button

def custom_exc(shell, etype, evalue, tb, tb_offset=None, ):
    """
    A customize exception method
    That we can assign to the ishell kernel
    Arguments follow the format of default exeception function
    """
    # gathering data on this error
    # the colorful traceback
    stb = itb.structured_traceback(etype, evalue, tb)
    sstb = itb.stb2text(stb)
    
    # the plain string of traceback
    traceback_string = format_exc()
    
    # input_history, sanitized(escape) for html
    input_history = list(html.escape(i)
                for i in ishell.history_manager.input_hist_parsed[-20:])
    
    # now time stamp
    now_full = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    now = datetime.now().strftime("%m%d_%H%M%S")
    
    error_data = json.dumps({
        "error_type_name": etype.__name__,
        "error_value":str(evalue),
        "traceback_string":html.escape(traceback_string),
        "timestamp":now_full,
        "input_history":input_history,
    }, indent=2)
    
    # create an error report in html format
    # by rendering a jinja2 template with error_data
    with open(STATIC/"html"/"bug"/"error_report.html","r") as f:
        temp = Template(f.read())
    
    error_report_page = temp.render(
        data = json.dumps(
            error_data,
        ))
    
    # create a mini error panel
    # a download button with embedded data
    download_button = render_download_button(
        error_report_page.encode(),
        filename=f"npakai_{etype.__name__}_{now}.html",
        description="🦋 Download Report",
        color="success")
    
    with open(STATIC/"html"/"bug"/"error_tiny_page.html", "r") as f:
        temp2 = Template(f.read())
        error_tiny_page = temp2.render(
            download_button=download_button,
            error_type_name=etype.__name__,
            error_value=str(evalue),
        )
    display(HTML(error_tiny_page))
    
    
    print(sstb)

## Assign our customized funtion

In [19]:
# export
ishell = get_ipython()
ishell.set_custom_exc((Exception,), custom_exc)

In [20]:
# hide
import slowai

Traceback (most recent call last):
  File "/var/folders/1d/fd4ndy_911b7z9n9ddf96fyh0000gn/T/ipykernel_84508/3296851057.py", line 2, in <module>
    import slowai
ModuleNotFoundError: No module named 'slowai'

